# How to read data from BigQuery

This notebook demonstrates two ways to use BigQuery with Python
1. by using SQL via [pandas-gbq](https://pandas-gbq.readthedocs.io/en/latest/)
2. by using only Python code to extract the data of interest from BigQuery via [Ibis](https://docs.ibis-project.org/)

## Setup

In [1]:
!pip3 install ibis-framework

  Using cached ibis_framework-1.4.0-py3-none-any.whl (647 kB)
  Using cached pandas-1.2.4-cp37-cp37m-manylinux1_x86_64.whl (9.9 MB)
  Using cached toolz-0.11.1-py3-none-any.whl (55 kB)
  Using cached pytz-2021.1-py2.py3-none-any.whl (510 kB)
  Using cached regex-2021.4.4-cp37-cp37m-manylinux2014_x86_64.whl (720 kB)
  Using cached multipledispatch-0.6.0-py3-none-any.whl (11 kB)
  Using cached numpy-1.20.3-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.3 MB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Using cached python_dateutil-2.8.1-py2.py3-none-any.whl (227 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-cpu 2.4.1 requires numpy~=1.19.2, but you have numpy 1.20.3 which is incompatible.
tensorflow-cpu 2.4.1 requires six~=1.15.0, but you have six 1.16.0 which is incompatible.
tensorflow-cpu 2.4.1 requires typing-extens

In [2]:
import pandas as pd
import pandas_gbq
import ibis
import os

In [3]:
BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']

## Option 1: Retrieve filtered data from BigQuery using SQL.

The following SQL will read a subset of columns and subset of rows from a BigQuery table into a Pandas dataframe.
* [Pandas](http://pandas.pydata.org/pandas-docs/stable/) is a popular Python package for data manipulation.
* To learn more about SQL syntax see the [BigQuery standard SQL reference](https://cloud.google.com/bigquery/docs/reference/standard-sql/).

In [4]:
sample_info = pd.read_gbq("""
SELECT
  Sample,
  Gender,
  Relationship,
  Population,
  Population_Description,
  Super_Population,
  Super_Population_Description,
  Total_Exome_Sequence,
  Main_Project_E_Platform,
  Main_Project_E_Centers
FROM
  `bigquery-public-data.human_genome_variants.1000_genomes_sample_info`
WHERE
  -- Only include information for samples in phase 1.
  In_Phase1_Integrated_Variant_Set = TRUE
""")

In [5]:
sample_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1092 entries, 0 to 1091
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Sample                        1092 non-null   object 
 1   Gender                        1092 non-null   object 
 2   Relationship                  1092 non-null   object 
 3   Population                    1092 non-null   object 
 4   Population_Description        1092 non-null   object 
 5   Super_Population              1092 non-null   object 
 6   Super_Population_Description  1092 non-null   object 
 7   Total_Exome_Sequence          1069 non-null   float64
 8   Main_Project_E_Platform       1092 non-null   object 
 9   Main_Project_E_Centers        1092 non-null   object 
dtypes: float64(1), object(9)
memory usage: 85.4+ KB


In [6]:
sample_info.describe()

,Total_Exome_Sequence
count,1.069000e+03
mean,1.028243e+10
std,3.845551e+09
min,2.477265e+09
25%,8.078811e+09
50%,9.730942e+09
75%,1.142406e+10
max,3.511709e+10


In [7]:
sample_info.head()

,Sample,Gender,Relationship,Population,Population_Description,Super_Population,Super_Population_Description,Total_Exome_Sequence,Main_Project_E_Platform,Main_Project_E_Centers
0,HG00377,female,,FIN,Finnish in Finland,EUR,European,4.105720e+09,ILLUMINA,BI
1,HG01278,female,mother,CLM,"Colombian in Medellin, Colombia",AMR,American,7.717205e+09,ILLUMINA,BCM
2,NA18527,female,,CHB,"Han Chinese in Bejing, China",EAS,East Asian,5.811252e+09,ILLUMINA,BCM
3,NA19672,female,mother,MXL,"Mexican Ancestry in Los Angeles, California",AMR,American,NaN,,
4,NA20363,female,mother,ASW,African Ancestry in Southwest US,AFR,African,NaN,,


## Option 2: Retrieve filtered data from BigQuery using Python.

The following Python code will read a BigQuery table into a Pandas dataframe.

From https://cloud.google.com/community/tutorials/bigquery-ibis

*[Ibis](http://ibis-project.org/) is a Python library for doing data analysis. It offers a Pandas-like environment for executing data analysis in big data processing systems such as Google BigQuery. Ibis's primary goals are to be a type safe, expressive, composable, and familiar replacement for SQL.*

In [8]:
conn = ibis.bigquery.connect(
    project_id=BILLING_PROJECT_ID,
    dataset_id='bigquery-public-data.human_genome_variants')

In [9]:
print('ibis version: %s' % ibis.__version__)

ibis version: 1.4.0


In [10]:
sample_info_tbl = conn.table('1000_genomes_sample_info')
sample_info_tbl

BigQueryTable[table]
  name: bigquery-public-data.human_genome_variants.1000_genomes_sample_info
  schema:
    Sample : string
    Family_ID : string
    Population : string
    Population_Description : string
    Gender : string
    Relationship : string
    Unexpected_Parent_Child : string
    Non_Paternity : string
    Siblings : string
    Grandparents : string
    Avuncular : string
    Half_Siblings : string
    Unknown_Second_Order : string
    Third_Order : string
    In_Low_Coverage_Pilot : boolean
    LC_Pilot_Platforms : string
    LC_Pilot_Centers : string
    In_High_Coverage_Pilot : boolean
    HC_Pilot_Platforms : string
    HC_Pilot_Centers : string
    In_Exon_Targetted_Pilot : boolean
    ET_Pilot_Platforms : string
    ET_Pilot_Centers : string
    Has_Sequence_in_Phase1 : boolean
    Phase1_LC_Platform : string
    Phase1_LC_Centers : string
    Phase1_E_Platform : string
    Phase1_E_Centers : string
    In_Phase1_Integrated_Variant_Set : boolean
    Has_Phase1_chr

In [11]:
# Define the filter criteria.
phase_1_only = sample_info_tbl.In_Phase1_Integrated_Variant_Set == True

# Apply the filter and choose the columns to return.
phase_1_sample_info_tbl = sample_info_tbl.filter(phase_1_only)['Sample', 'Gender', 'Relationship', 'Population', 'Population_Description',
                'Super_Population', 'Super_Population_Description', 'Total_Exome_Sequence',
                'Main_Project_E_Platform', 'Main_Project_E_Centers']

In [12]:
# Optional: take a look at the SQL.
print(phase_1_sample_info_tbl.compile())

SELECT `Sample`, `Gender`, `Relationship`, `Population`,
       `Population_Description`, `Super_Population`,
       `Super_Population_Description`, `Total_Exome_Sequence`,
       `Main_Project_E_Platform`, `Main_Project_E_Centers`
FROM (
  SELECT *
  FROM `bigquery-public-data.human_genome_variants.1000_genomes_sample_info`
  WHERE `In_Phase1_Integrated_Variant_Set` = TRUE
) t0


In [13]:
# Optional: See how much data this will return.
phase_1_sample_info_tbl.count().execute()

/usr/local/lib/python3.7/dist-packages/google/cloud/bigquery/client.py:462: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


1092

In [14]:
# Go ahead and retrieve the data.
phase_1_sample_info_df = phase_1_sample_info_tbl.limit(1000000).execute()
phase_1_sample_info_df.shape

(1092, 10)

In [15]:
phase_1_sample_info_df.head()

,Sample,Gender,Relationship,Population,Population_Description,Super_Population,Super_Population_Description,Total_Exome_Sequence,Main_Project_E_Platform,Main_Project_E_Centers
0,HG00377,female,,FIN,Finnish in Finland,EUR,European,4.105720e+09,ILLUMINA,BI
1,HG01278,female,mother,CLM,"Colombian in Medellin, Colombia",AMR,American,7.717205e+09,ILLUMINA,BCM
2,NA18527,female,,CHB,"Han Chinese in Bejing, China",EAS,East Asian,5.811252e+09,ILLUMINA,BCM
3,NA19672,female,mother,MXL,"Mexican Ancestry in Los Angeles, California",AMR,American,NaN,,
4,NA20363,female,mother,ASW,African Ancestry in Southwest US,AFR,African,NaN,,


# Provenance

In [16]:
import datetime
print(datetime.datetime.now())

2021-05-19 23:15:19.915762


In [17]:
!pip3 freeze

absl-py==0.12.0
anyio==3.1.0
argon2-cffi==20.1.0
arrow==1.1.0
arviz==0.11.2
asn1crypto==0.24.0
astroid==2.5.6
astunparse==1.6.3
async-generator==1.10
attrs==21.2.0
backcall==0.2.0
bagit==1.8.1
bgzip==0.3.5
binaryornot==0.4.4
biopython==1.78
bleach==3.3.0
bokeh==2.3.1
brewer2mpl==1.4.1
bx-python==0.8.11
CacheControl==0.11.7
cachetools==4.2.2
certifi==2020.12.5
cffi==1.14.5
cftime==1.4.1
chardet==4.0.0
cli-builder==0.1.5
click==7.1.2
colorama==0.4.4
confuse==1.4.0
cookiecutter==1.7.2
crcmod==1.7
cryptography==3.4.7
cwltool==1.0.20190228155703
cycler==0.10.0
Cython==0.29.23
decorator==4.4.2
defusedxml==0.7.1
descartes==1.1.0
dill==0.3.3
entrypoints==0.3
facets-overview==1.0.0
fastinterval==0.1.1
fastprogress==1.0.0
filelock==3.0.12
firecloud==0.16.25
flatbuffers==1.12
future==0.18.2
gast==0.3.3
ggplot==0.11.5
gitdb==4.0.7
GitPython==3.1.17
google-api-core==1.26.3
google-auth==1.30.0
google-auth-oauthlib==0.4.4
google-cloud-bigquery==2.15.0
google-cloud-bigquery-datatransfer==3.1.1
google-

Copyright 2018 The Broad Institute, Inc., Verily Life Sciences, LLC All rights reserved.

This software may be modified and distributed under the terms of the BSD license. See the LICENSE file for details.